<a href="https://colab.research.google.com/github/anaghasid/GenPruning/blob/main/T5_Pruning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet datasets
!pip install --quiet evaluate
!pip install --quiet accelerate
# !pip install --quiet bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.4 MB/s eta 0:00:00


In [2]:
import numpy as np
import torch
import evaluate
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from datasets import load_dataset
import accelerate
from sklearn.metrics import accuracy_score
import time
from torch.utils.data import DataLoader
from collections import defaultdict
import pandas as pd
import random

In [3]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

In [4]:
def make_model():
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  # model = AutoModelForSequenceClassification.from_pretrained(model_name, trust_remote_code=True,
  #                                                          device_map=device)
  model = DistilBertForSequenceClassification.from_pretrained(model_name, device_map=device)
  return model


tokenizer = DistilBertTokenizer.from_pretrained(model_name)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.8 MB/s eta 0:00:00


In [7]:
# Load a pretrained model and tokenizer
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, # enable 4-bit quantization
    bnb_4bit_quant_type = 'nf4', # information theoretically optimal dtype for normally distributed weights
    bnb_4bit_use_double_quant = True, # quantize quantized weights //insert xzibit meme
    bnb_4bit_compute_dtype = torch.bfloat16 # optimized fp format for ML
)


In [8]:
model = make_model()

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [10]:
print("Number of encoders/decoders:", model.config.n_layers)
print("Number of heads:", model.config.n_heads)

Number of encoders/decoders: 6
Number of heads: 12


In [11]:
# Load the GLUE dataset (SST-2 as an example)
# dataset = load_dataset("glue", "cola", split='validation', trust_remote_code=True)
dataset = load_dataset("stanfordnlp/sst2", split="train", trust_remote_code=True)
print("Sample from the dataset:", dataset[0])
# metric = evaluate.load("glue", "cola", trust_remote_code=True)

README.md:   0%|          | 0.00/5.27k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Sample from the dataset: {'idx': 0, 'sentence': 'hide new secretions from the parental units ', 'label': 0}


In [12]:
print(len(dataset))
dataset[0:10]

67349


{'idx': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'sentence': ['hide new secretions from the parental units ',
  'contains no wit , only labored gags ',
  'that loves its characters and communicates something rather beautiful about human nature ',
  'remains utterly satisfied to remain the same throughout ',
  'on the worst revenge-of-the-nerds clichés the filmmakers could dredge up ',
  "that 's far too tragic to merit such superficial treatment ",
  'demonstrates that the director of such hollywood blockbusters as patriot games can still turn out a small , personal film with an emotional wallop . ',
  'of saucy ',
  "a depressed fifteen-year-old 's suicidal poetry ",
  "are more deeply thought through than in most ` right-thinking ' films "],
 'label': [0, 0, 1, 0, 0, 0, 1, 1, 0, 1]}

In [13]:
# Preprocess the data
def preprocess_function(examples):
    tokenized_inputs = tokenizer(examples['sentence'], truncation=True, padding='max_length', max_length=128)
    tokenized_inputs['label'] = examples['label']
    return tokenized_inputs

encoded_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

In [15]:
encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

dataloader = DataLoader(encoded_dataset, batch_size=16)


# Display a sample batch from the dataloader
for batch in dataloader:
    # print(batch['label'])
    input_ids = batch['input_ids'].tolist()
    attention_masks = batch['attention_mask'].tolist()
    labels = batch['label'].tolist()

    truncated_input_ids = [ids[:10] for ids in input_ids]  # Show only the first 10 tokens
    truncated_attention_masks = [masks[:10] for masks in attention_masks]

    df = pd.DataFrame({
        'Input IDs': truncated_input_ids,
        'Attention Mask': truncated_attention_masks,
        'Label': labels
    })

    print(df)
    break


                                                        Input IDs                  Attention Mask  Label
0     [101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]      0
1   [101, 3397, 2053, 15966, 1010, 2069, 4450, 2098, 18201, 2015]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]      0
2    [101, 2008, 7459, 2049, 3494, 1998, 10639, 2015, 2242, 2738]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]      1
3     [101, 3464, 12580, 8510, 2000, 3961, 1996, 2168, 2802, 102]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]      0
4     [101, 2006, 1996, 5409, 7195, 1011, 1997, 1011, 1996, 1011]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]      0
5    [101, 2008, 1005, 1055, 2521, 2205, 13800, 2000, 7857, 2107]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]      0
6   [101, 16691, 2008, 1996, 2472, 1997, 2107, 5365, 27858, 2015]  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]      1
7                 [101, 1997, 7842, 14194, 2100, 102, 0, 0, 0, 0]  [1, 1, 1, 1, 1, 1, 0, 0, 0, 0]      1
8    [101, 1037, 14777, 5417, 1011, 2095, 1011, 2214, 1

In [35]:
# Modify model based on chromosome
def modify_model(model, chromosome):
    num_heads = model.config.n_heads
    num_blocks = model.config.n_layers
    print(num_blocks)
    # num_ffn = model.config.num_hidden_layers

    # Disable attention heads
    heads_to_prune = defaultdict(list)
    for i, gene in enumerate(chromosome):
        if gene == 0:
          block_num = i//num_heads
          head_num = i%num_heads
          heads_to_prune[block_num].append(head_num)

    print(heads_to_prune)
    if heads_to_prune:
      if hasattr(model, "distilbert"):
          for block_num in heads_to_prune:
              # Access the correct layers for DistilBERT
              model.distilbert.transformer.layer[block_num].attention.prune_heads(heads_to_prune[block_num])
      # Check if the model has a bert layer (for BERT-based models)
      elif hasattr(model, "bert"):
          for block_num in heads_to_prune:
              model.bert.encoder.layer[block_num].attention.prune_heads(heads_to_prune[block_num])

      else:
        raise ValueError("Unsupported model architecture. The model should have either a 'distilbert' or 'bert' layer.")





        # for block_num in heads_to_prune:
        #     model.transformer.encoder.block[block_num].layer[0].SelfAttention.prune_heads(heads_to_prune[block_num])
        #     # if(bloack_num!=0):
        #     model.transformer.decoder.block[block_num].layer[0].SelfAttention.prune_heads(heads_to_prune[block_num])
            # model.transformer.decoder.block[block_num].layer[1].EncDecAttention.prune_heads(heads_to_prune[block_num])

            # print(block_num, model.transformer.encoder.block[block_num].layer[0].SelfAttention.n_heads)
            # model.prune_heads(heads_to_prune)

            # model.transformer.encoder.block[block_num].layer[0].SelfAttention.q = prune_linear_layer(model.transformer.encoder.block[block_num].layer[0].SelfAttention.q, index)
            # model.transformer.encoder.block[block_num].layer[0].SelfAttention.k = prune_linear_layer(model.transformer.encoder.block[block_num].layer[0].SelfAttention.k, index)
            # model.transformer.encoder.block[block_num].layer[0].SelfAttention.v = prune_linear_layer(model.transformer.encoder.block[block_num].layer[0].SelfAttention.v, index)
            # model.transformer.encoder.block[block_num].layer[0].SelfAttention.o = prune_linear_layer(model.transformer.encoder.block[block_num].layer[0].SelfAttention.o, index, dim=1)

In [33]:
def evaluate_fitness(chromosome, model, encoded_dataset):
    # Modify the model according to the chromosome
    if len(chromosome):
      model = make_model()
      modify_model(model, chromosome)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    start_time = time.time()
    predictions = []
    labels = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)

            # Get predictions
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1)
            predictions.extend(preds.cpu().numpy())
            labels.extend(batch['label'].cpu().numpy())


    # Evaluate the model
    print("For the chromosome", chromosome)
    fitness = accuracy_score(labels, predictions)
    print(f"Validation Accuracy: {fitness:.4f}")
    end_time = time.time()
    print("Execution time = ",end_time-start_time)
    print()
    return fitness

In [46]:
# without modifying the model
print(evaluate_fitness([], model, encoded_dataset))

For the chromosome []
Validation Accuracy: 0.9886
Execution time =  267.57011675834656

0.9885521685548412


In [29]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [30]:
def create_random_binary_list(length, percentage_of_zeros):
    # Calculate the number of zeros and ones based on the percentage
    num_zeros = int(length * (percentage_of_zeros / 100))
    num_ones = length - num_zeros

    # Create the list with the required number of 0s and 1s
    binary_list = [0] * num_zeros + [1] * num_ones

    # Shuffle the list to randomize the order
    random.shuffle(binary_list)

    return binary_list

def initialize_chromosome(num_genes):
  # initialize chromosome with given sparsity percentage
  return create_random_binary_list(num_genes, 30)

In [22]:
# checking to see if decoder is there
print(model.config.is_decoder)

False


In [36]:
model  = make_model()
chromosome = initialize_chromosome(model.config.n_heads*model.config.n_layers)
print("Chromosome:", chromosome)
fitness = evaluate_fitness(chromosome, model, encoded_dataset)
print("Fitness score (accuracy):", fitness)


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Chromosome: [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1]
6
defaultdict(<class 'list'>, {0: [9], 1: [1, 2, 8, 9], 2: [0, 7, 9, 10], 3: [2, 3, 4], 4: [2, 4, 5, 6, 11], 5: [2, 3, 8, 10]})
For the chromosome [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1]
Validation Accuracy: 0.9842
Execution time =  240.31958293914795

Fitness score (accuracy): 0.9842165436754814


In [37]:
def find_size(model):
  total_size_in_bytes = sum(p.numel() * p.element_size() for p in model.parameters())
  total_size_in_megabytes = total_size_in_bytes / (1024 ** 2)
  print(f"Model size: {total_size_in_megabytes:.2f} MB")

In [ ]:
# m  = AutoModelForSequenceClassification.from_pretrained(model_name, trust_remote_code=True, device_map=device)
# print(find_size(m), find_size(model))

In [38]:
POPN_SIZE = 10
crossover_rate = 0.7
mutation_rate = 0.1

In [39]:
def initialize_population(chromosome_length):
  # initialize random population
  population = []
  for _ in range(POPN_SIZE):
    chromosome = initialize_chromosome(chromosome_length)
    population.append(chromosome)
  return population

In [ ]:
# Selection (Tournament Selection)
'''
def select_parents(population, fitness_scores):
    parents = []
    for _ in range(POPN_SIZE):
        tournament = np.random.choice(POPN_SIZE, 2)
        print("tournament = ",tournament)
        winner = tournament[np.argmax(fitness_scores[tournament])]
        parents.append(population[winner])
    return np.array(parents)
    '''

In [40]:
# roulette wheel selection
def select_parents(population, fitness_scores, num_parents):
    # Normalize fitness scores to create a probability distribution
    total_fitness = np.sum(fitness_scores)
    probabilities = fitness_scores / total_fitness

    # Select parents based on their fitness proportion (roulette wheel selection)
    # selected_parents = np.random.choice(population, size=num_parents, p=probabilities, replace=True)
    selected_parents = random.choices(population, weights=probabilities, k=num_parents)


    return np.array(selected_parents)

In [41]:
# Crossover (Single-point crossover)
def crossover(parent1, parent2):
    if np.random.rand() < crossover_rate:
        point = np.random.randint(1, len(parent1) - 1)
        child1 = np.concatenate([parent1[:point], parent2[point:]])
        child2 = np.concatenate([parent2[:point], parent1[point:]])
    else:
        child1, child2 = parent1, parent2
    return child1, child2

In [42]:
# Mutation (Flip bit mutation)
def mutate(chromosome):
    for i in range(len(chromosome)):
        if np.random.rand() < mutation_rate:
            chromosome[i] = 1 - chromosome[i]
    return chromosome

In [43]:
def elitism_and_selection(population, fitness_scores, num_elites, num_parents):
    # Elitism: Keep the top num_elites individuals
    elite_indices = np.argsort(fitness_scores)[-num_elites:]  # Get indices of top individuals
    elites = [population[i] for i in elite_indices]

    # Perform roulette wheel selection for the rest of the parents
    remaining_population = np.delete(population, elite_indices, axis=0)
    remaining_fitness_scores = np.delete(fitness_scores, elite_indices)

    num_to_select = num_parents - num_elites
    selected_parents = select_parents(remaining_population, remaining_fitness_scores, num_to_select)

    # Combine elites and selected parents
    next_generation = np.vstack((elites, selected_parents))

    return next_generation

In [ ]:
'''def genetic_algorithm(model, num_generations, desired_sparsity):
  population = initialize_population(model.config.num_heads*model.config.num_layers)
  for generation in range(num_generations):
      fitness_scores = np.array([evaluate_fitness(chrom, model, encoded_dataset, metric) for chrom in population])
      parents = select_parents(population, fitness_scores)
      new_population = []
      for i in range(0, POPN_SIZE, 2):
          parent1, parent2 = parents[i], parents[i + 1]
          child1, child2 = crossover(parent1, parent2)
          child1 = mutate(child1)
          child2 = mutate(child2)
          new_population.extend([child1, child2])
      population = np.array(new_population)

      # Check for desired sparsity level
      sparsity_levels = np.mean(population == 0, axis=1)
      print(sparsity_levels)
      if np.any(sparsity_levels >= desired_sparsity):
          best_chromosome = population[np.argmax(sparsity_levels)]
          break
  return best_chromosome'''

In [44]:
def genetic_algorithm(model, num_generations, desired_sparsity):
  population = initialize_population(model.config.n_heads*model.config.n_layers)  # Initialize the population
  for generation in range(num_generations):
      fitness_scores = np.array([evaluate_fitness(chrom, model, encoded_dataset) for chrom in population])
      best_chromosome = population[np.argmax(fitness_scores)]
      print("new fitness scores:", fitness_scores)
      print(f"best chromosome in generation {generation} is {best_chromosome} with accuracy {fitness_scores[np.argmax(fitness_scores)]}")
      parents = elitism_and_selection(population, fitness_scores, 4, POPN_SIZE)
      # parents = select_parents(population, fitness_scores, POPN_SIZE)
      new_population = []
      for i in range(0, POPN_SIZE, 2):
          parent1, parent2 = parents[i], parents[i + 1]
          child1, child2 = crossover(parent1, parent2)
          child1 = mutate(child1)
          child2 = mutate(child2)
          new_population.extend([child1, child2])
      population = np.array(new_population)
  return best_chromosome

In [ ]:
'''genetic_algorithm(model, 10, 0.3)'''

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google-t5/t5-small and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


6
defaultdict(<class 'list'>, {0: [0, 1, 4], 1: [2, 5, 6, 7], 2: [1], 3: [4, 6], 4: [1, 7], 5: [1, 7]})
Pruning
Pruning
Pruning
Pruning
Pruning
Pruning
For the chromosome [0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0]
Validation Accuracy: 0.3538
Execution time =  220.13971972465515



Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google-t5/t5-small and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


6
defaultdict(<class 'list'>, {0: [5], 1: [3, 4, 5, 6], 2: [4, 7], 4: [2, 3, 4, 5], 5: [1, 5, 7]})
Pruning
Pruning
Pruning
Pruning
Pruning
For the chromosome [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0]
Validation Accuracy: 0.3730
Execution time =  222.45740389823914



Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google-t5/t5-small and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google-t5/t5-small and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


6
defaultdict(<class 'list'>, {0: [5, 6], 1: [3, 4], 2: [3, 5, 6], 3: [0, 5], 4: [1, 6, 7], 5: [3, 7]})
Pruning
Pruning
Pruning
Pruning
Pruning
Pruning
For the chromosome [1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0]
Validation Accuracy: 0.3087
Execution time =  211.41342115402222



Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google-t5/t5-small and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


6
defaultdict(<class 'list'>, {0: [1, 7], 1: [7], 2: [2, 3, 7], 3: [1, 3, 5, 6], 4: [4], 5: [2, 3, 6]})
Pruning
Pruning
Pruning
Pruning
Pruning
Pruning
For the chromosome [1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1]
Validation Accuracy: 0.6913
Execution time =  212.1215159893036

tournament =  [5 5]
tournament =  [ 1 19]
tournament =  [13 19]
tournament =  [2 6]
tournament =  [8 7]
tournament =  [18 11]
tournament =  [16 16]
tournament =  [12  4]
tournament =  [10 15]
tournament =  [3 1]
tournament =  [2 5]
tournament =  [6 8]
tournament =  [14 14]
tournament =  [6 5]
tournament =  [ 2 15]
tournament =  [ 4 17]
tournament =  [15  7]
tournament =  [12 16]
tournament =  [15  5]
tournament =  [ 3 12]
[0.35416667 0.3125     0.33333333 0.22916667 0.33333333 0.41666667
 0.3125     0.39583333 0.25       0.35416667 0.3125     0.33333333
 0.35416667 0.33333333 0.375      0.41666667 0.29166667 0.2

array([0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 1])

In [45]:
genetic_algorithm(model, 10, 0.3)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


6
defaultdict(<class 'list'>, {0: [0, 1, 5, 7, 9, 10], 1: [2, 8], 2: [6, 8, 11], 3: [3, 6, 7, 8], 4: [3, 4, 5, 11], 5: [6, 9]})


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


For the chromosome [0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1]
Validation Accuracy: 0.9819
Execution time =  237.35089755058289

6
defaultdict(<class 'list'>, {0: [1, 2, 4, 6, 10], 1: [0, 1, 4, 5], 2: [0], 3: [0, 1, 4, 9, 10, 11], 4: [1, 5, 8, 11], 5: [2]})


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


...

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


For the chromosome [1 1 0 0 1 1 0 0 0 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 0 1 0 1 0 1 1 1 0 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 0 1 0 1 1 0 1 1 1 0 0 1 1 0 0 1 0 1]
Validation Accuracy: 0.9826
Execution time =  240.13011121749878

6
defaultdict(<class 'list'>, {0: [0, 4, 5, 7, 9, 10, 11], 1: [2, 3, 5, 8], 2: [0, 6, 8], 3: [6, 7, 8], 4: [3, 4, 5, 11], 5: [0, 5, 6, 9]})


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


For the chromosome [0 1 1 1 0 0 1 0 1 0 0 0 1 1 0 0 1 0 1 1 0 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1 1
 1 1 1 1 1 0 0 0 1 1 1 1 1 1 0 0 0 1 1 1 1 1 0 0 1 1 1 1 0 0 1 1 0 1 1]
Validation Accuracy: 0.9821
Execution time =  234.93710231781006

6
defaultdict(<class 'list'>, {0: [1, 9], 1: [3, 6, 8, 10], 2: [3, 7, 9], 3: [1, 3, 4, 6], 4: [0, 5, 6, 8], 5: [5, 8, 9]})


KeyboardInterrupt: 